In [1]:
# Import libraries
import multiprocessing
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from datetime import date, timedelta                                                                                                               
import oceanbench
import os
import pandas as pd


In [2]:
gnet_files = sorted([os.path.join('/data/data/toeval/glonet/', f) for f in os.listdir('/data/data/toeval/glonet') if f.endswith(".nc")])
gnetset = [xr.open_dataset(f) for f in gnet_files]
grys_files = sorted([os.path.join('/data/data/GLOREF/GLONET/', f) for f in os.listdir('/data/data/GLOREF/GLONET/') if f.endswith(".nc")])
grysset = [xr.open_dataset(f) for f in grys_files]


In [14]:
#### in rmse remove depth from zos
vars_eval = oceanbench.metrics.rmse(
    glonet_datasets=gnetset,
    glorys_datasets=grysset,
)

depth=0
depth=1
depth=2
depth=3
depth=4
depth=5
depth=6
depth=7
depth=8
depth=9
depth=10
depth=11
depth=12
depth=13
depth=14
depth=15
depth=16
depth=17
depth=18
depth=19
depth=20


In [15]:
uo=np.array(vars_eval.item()['uo'])
vo=np.array(vars_eval.item()['vo'])
thetao=np.array(vars_eval.item()['thetao'])
so=np.array(vars_eval.item()['so'])
zos=np.array(vars_eval.item()['zos'])

In [16]:
from IPython.display import display, HTML

display(HTML('<h1 style="color:red; text-align:center;">Surface uo Score</h1>'))
df = pd.DataFrame([["Lead Day " + str(i+1) for i in range(10)], uo[0,:]])
df.index = ["", "Score"]
df.style.set_properties(**{"border": "1px solid black", "text-align": "center"})


In [17]:


display(HTML('<h1 style="color:red; text-align:center;">50m uo Score</h1>'))
df = pd.DataFrame([["Lead Day " + str(i+1) for i in range(10)], uo[1,:]])
df.index = ["", "Score"]
df.style.set_properties(**{"border": "1px solid black", "text-align": "center"})

In [18]:
display(HTML('<h1 style="color:red; text-align:center;">Surface vo Score</h1>'))
df = pd.DataFrame([["Lead Day " + str(i+1) for i in range(10)], vo[0,:]])
df.index = ["", "Score"]
df.style.set_properties(**{"border": "1px solid black", "text-align": "center"})

In [19]:


display(HTML('<h1 style="color:red; text-align:center;">50m vo Score</h1>'))
df = pd.DataFrame([["Lead Day " + str(i+1) for i in range(10)], uo[1,:]])
df.index = ["", "Score"]
df.style.set_properties(**{"border": "1px solid black", "text-align": "center"})

In [20]:
display(HTML('<h1 style="color:red; text-align:center;">Surface thetao Score</h1>'))
df = pd.DataFrame([["Lead Day " + str(i+1) for i in range(10)], thetao[0,:]])
df.index = ["", "Score"]
df.style.set_properties(**{"border": "1px solid black", "text-align": "center"})

In [21]:


display(HTML('<h1 style="color:red; text-align:center;">50m thetao Score</h1>'))
df = pd.DataFrame([["Lead Day " + str(i+1) for i in range(10)], thetao[1,:]])
df.index = ["", "Score"]
df.style.set_properties(**{"border": "1px solid black", "text-align": "center"})

In [22]:
display(HTML('<h1 style="color:red; text-align:center;">Surface so Score</h1>'))
df = pd.DataFrame([["Lead Day " + str(i+1) for i in range(10)], so[0,:]])
df.index = ["", "Score"]
df.style.set_properties(**{"border": "1px solid black", "text-align": "center"})

In [23]:


display(HTML('<h1 style="color:red; text-align:center;">50m so Score</h1>'))
df = pd.DataFrame([["Lead Day " + str(i+1) for i in range(10)], so[1,:]])
df.index = ["", "Score"]
df.style.set_properties(**{"border": "1px solid black", "text-align": "center"})

In [73]:
display(HTML('<h1 style="color:red; text-align:center;"> zos Score</h1>'))
df = pd.DataFrame([["Lead Day " + str(i+1) for i in range(10)], zos[0,:]])
df.index = ["", "Score"]
df.style.set_properties(**{"border": "1px solid black", "text-align": "center"})

### DERIVED QUANTITIES EVALUATION


### HERE START PROCESS ORTIENTED EVAL:

for this part we should call the functions from derived_quantities mainly mld and geostrophic_currents for glonet_dataset and glorys_dataset(see bellow), this will generate new dwtasets including MLD and g_currents. we should do this for the whole time series (e.g see above). Then redo the same thing we did for zos uo vo ..
create new tables for MLD, u_geo and v_geo

In [1]:
## removed [lead ] from core.geo and core.mld also core geo_ add time dimension

for i in range(len(gnetset)):
    print(i)
    gnetset[i] = oceanbench.derived_quantities.mld(
        dataset=gnetset[i],
        lead=1,#not used
    )
    gnetset[i] = oceanbench.derived_quantities.geostrophic_currents(
        dataset=gnetset[i],
        lead=1, ##not used
        variable="zos",
    )
    grysset[i] = oceanbench.derived_quantities.mld(
        dataset=grysset[i],
        lead=1,#not used
    )
    grysset[i] = oceanbench.derived_quantities.geostrophic_currents(
        dataset=grysset[i],
        lead=1, ##not used
        variable="zos",
    )
#gnet now has MLD, u_geo, and v_geo vars, we should call derived_quantities_eval = oceanbench.metrics.rmse(glonet_datasets=gnetset,glorys_datasets=grysset,), but this time we should explicitly tell the function we want MLD/u_geo/v_geo

In [4]:

##################
### as you can see here I copy pasted the rmse funcrion cause it has vars already predefined, it is the same but with derived quantities. 

from typing import Any, List
import xarray
import numpy
def get_rmse_glonet(forecast, ref, var, lead, level):
    cpu_count = multiprocessing.cpu_count()
    with multiprocessing.Pool(cpu_count) as _:
        if var == "MLD" or var == "v_geo" or var == "u_geo" :
            mask = (~numpy.isnan(ref[var].data[lead]) )& (~numpy.isnan(forecast[var].data[lead]))
            rmse = numpy.sqrt(numpy.nanmean((forecast[var][lead].data[mask] - ref[var][ lead].data[mask]) ** 2))
        else:
            mask = ~numpy.isnan(forecast[var][lead, level].data) & ~numpy.isnan(ref[var][lead, level].data)
            rmse = numpy.sqrt(
                numpy.mean((forecast[var][lead, level].data[mask] - ref[var][lead, level].data[mask]) ** 2)
            )
    return rmse


def get_glonet_rmse_for_given_days(
    depthg,
    var,
    glonet_datasets: List[xarray.Dataset],
    glorys_datasets: List[xarray.Dataset],
):
    j = 0
    nweeks = 1
    aa = numpy.zeros((nweeks, 10))

    for glonet, glorys in zip(glonet_datasets, glorys_datasets):
        for i in range(0, 10):
            aa[j, i] = get_rmse_glonet(glonet, glorys, var, i, depthg)
        j = j + 1
        if j > nweeks - 1:
            break
    glonet_rmse = aa.mean(axis=0)
    return glonet_rmse


def glonet_pointwise_evaluation_core(
    glonet_datasets: List[xarray.Dataset],
    glorys_datasets: List[xarray.Dataset],
) -> numpy.ndarray[Any]:
    gnet = {"u_geo": [], "v_geo": [], "MLD": []}
    variables_withouth_zos = ["u_geo", "v_geo", "MLD"]
    for variable in variables_withouth_zos:
        gnet[variable].append(
                get_glonet_rmse_for_given_days(
                    1,
                    variable,
                    glonet_datasets,
                    glorys_datasets,
                )
            )
    return numpy.array(gnet)

In [5]:
##########here we explude the equator
for i in range(len(gnetset)):
    gnetset[i]['u_geo'] = gnetset[i]['u_geo'].where((gnetset[i].lat < -0.5) | (gnetset[i].lat > 0.5), drop=False)
    gnetset[i]['v_geo'] = gnetset[i]['v_geo'].where((gnetset[i].lat < -0.5) | (gnetset[i].lat > 0.5), drop=False)


aa=glonet_pointwise_evaluation_core(
    glonet_datasets=gnetset,
    glorys_datasets=grysset,
)

In [6]:
u_geo=np.array(aa.item()['u_geo'])
v_geo=np.array(aa.item()['v_geo'])
mld=np.array(aa.item()['MLD'])


In [7]:
from IPython.display import display, HTML
display(HTML('<h1 style="color:red; text-align:center;"> MLD Score</h1>'))
df = pd.DataFrame([["Lead Day " + str(i+1) for i in range(10)], mld[0,:]])
df.index = ["", "Score"]
df.style.set_properties(**{"border": "1px solid black", "text-align": "center"})

In [8]:
from IPython.display import display, HTML
display(HTML('<h1 style="color:red; text-align:center;"> u_geo Score</h1>'))
df = pd.DataFrame([["Lead Day " + str(i+1) for i in range(10)], u_geo[0,:]])
df.index = ["", "Score"]
df.style.set_properties(**{"border": "1px solid black", "text-align": "center"})

In [9]:
from IPython.display import display, HTML
display(HTML('<h1 style="color:red; text-align:center;"> v_geo Score</h1>'))
df = pd.DataFrame([["Lead Day " + str(i+1) for i in range(10)], v_geo[0,:]])
df.index = ["", "Score"]
df.style.set_properties(**{"border": "1px solid black", "text-align": "center"})

### LAGRANGIAN TRAJ EVAL


we stop at 9d not to have particle leaving domain

In [10]:
### edit langrangian_analysis.py:
#####     --->lon = dataset.sel(lat=slice(latzone[0], latzone[1]), lon=slice(lonzone[0], lonzone[1])).lon.data
######    --->lat = dataset.sel(lat=slice(latzone[0], latzone[1]), lon=slice(lonzone[0], lonzone[1])).lat.data
####  remove thetao and zos from ds
#### new files included in the same folder as this jupy

######### lagrangian drifts in a small zone in NW africa, to be updated later to inlude other zones either averaging or defininf 4~5 major zonre for eval
euclidean_distance=[]
for i in range(len(gnetset)):
    euclidean_distance.append (oceanbench.metrics.euclidean_distance(
    first_dataset=gnetset[i],
    second_dataset=grysset[i],
    minimum_latitude=10,
    maximum_latitude=40,
    minimum_longitude=-40,
    maximum_longitude=0,))


start
INFO: Output files are stored in tst.zarr.
100%|████████████████████| 777600.0/777600.0 [00:04<00:00, 166003.62it/s]
<class 'numpy.ndarray'>
start
INFO: Output files are stored in tst.zarr.
100%|█████████████████████| 777600.0/777600.0 [00:07<00:00, 98577.32it/s]
<class 'numpy.ndarray'>
start
INFO: Output files are stored in tst.zarr.
100%|█████████████████████| 777600.0/777600.0 [00:08<00:00, 91075.98it/s]
<class 'numpy.ndarray'>
start
INFO: Output files are stored in tst.zarr.
100%|█████████████████████| 777600.0/777600.0 [00:10<00:00, 74644.05it/s]
<class 'numpy.ndarray'>
start
INFO: Output files are stored in tst.zarr.
100%|█████████████████████| 777600.0/777600.0 [00:08<00:00, 88841.84it/s]
<class 'numpy.ndarray'>
start
INFO: Output files are stored in tst.zarr.
100%|████████████████████| 777600.0/777600.0 [00:06<00:00, 114462.17it/s]
<class 'numpy.ndarray'>
start
INFO: Output files are stored in tst.zarr.
100%|████████████████████| 777600.0/777600.0 [00:06<00:00, 126291.75i

In [11]:
from IPython.display import display, HTML
lag_traj = np.mean(euclidean_distance, axis=0)
display(HTML('<h1 style="color:red; text-align:center;"> Lag-traj div [km ]</h1>'))
df = pd.DataFrame([["Lead Day " + str(i+1) for i in range(1, 9)], lag_traj[1:9]])
df.index = ["", "Score"]
df.style.set_properties(**{"border": "1px solid black", "text-align": "center"})